In [195]:
url = 'https://www.amazon.ca/product-reviews/B0BN72FYFG'

In [196]:
from selenium import webdriver

In [197]:
driver = webdriver.Chrome()
driver.get(url)
driver.implicitly_wait(3)

html_page = driver.page_source
driver.quit()


In [198]:
html_page

'<html lang="en-ca" class=" a-js a-audio a-video a-canvas a-svg a-drag-drop a-geolocation a-history a-webworker a-autofocus a-input-placeholder a-textarea-placeholder a-local-storage a-gradients a-hires a-transform3d a-touch-scrolling a-text-shadow a-text-stroke a-box-shadow a-border-radius a-border-image a-opacity a-transform a-transition null" data-19ax5a9jf="dingo" data-aui-build-date="3.24.3-2024-04-20"><!-- sp:feature:head-start --><head><script async="" src="https://images-na.ssl-images-amazon.com/images/I/31bJewCvY-L.js" crossorigin="anonymous"></script><script>var aPageStart = (new Date()).getTime();</script><meta charset="utf-8">\n<!-- sp:end-feature:head-start -->\n<!-- sp:feature:csm:head-open-part1 -->\n\n<script type="text/javascript">var ue_t0=ue_t0||+new Date();</script>\n<!-- sp:end-feature:csm:head-open-part1 -->\n<!-- sp:feature:cs-optimization -->\n<meta http-equiv="x-dns-prefetch-control" content="on">\n<link rel="dns-prefetch" href="https://images-na.ssl-images-ama

In [209]:
from bs4 import BeautifulSoup

In [210]:
soup = BeautifulSoup(html_page, 'lxml')

In [211]:
reviews = soup.findAll('span', attrs={'data-hook': True})
reviews = [review for review in reviews if review['data-hook'] == 'review-body']

In [212]:
for review in reviews:
    print(review)

<span class="a-size-base review-text review-text-content" data-hook="review-body">
<span>This is the second refurb iPhone that I have purchased over the years and am never disappointed in the quality of the product that meets or even exceeds actual Apple reburbished units.<br/>The phone was in excellent condition with not even a mark or a scratch anywhere to be seen. It even came with a charger and a cable (my 1st phone didn't).<br/>The battery life is definately better as well. The only wee issue I had was that I thought all iPhone 14's had the USB-C connecter but, alas, this one has the lightning plug instead. 'S OK, I'll live with it!</span>
</span>
<span class="a-size-base review-text review-text-content" data-hook="review-body">
<span>Deliberately waited for few days before giving my feedback. Got it for my daughter and she is very happy with the product, so no issues there but was expecting adapter along with the charging cable but didn't get one so was little disappointed. Still

In [213]:
len(review)

3